<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#神经网络层的函数实现" data-toc-modified-id="神经网络层的函数实现-1">神经网络层的函数实现</a></span></li><li><span><a href="#神经网络初始化" data-toc-modified-id="神经网络初始化-2">神经网络初始化</a></span><ul class="toc-item"><li><span><a href="#网络初始化" data-toc-modified-id="网络初始化-2.1">网络初始化</a></span></li><li><span><a href="#训练数据集初始化" data-toc-modified-id="训练数据集初始化-2.2">训练数据集初始化</a></span></li></ul></li><li><span><a href="#训练神经网络" data-toc-modified-id="训练神经网络-3">训练神经网络</a></span><ul class="toc-item"><li><span><a href="#设定加法问题" data-toc-modified-id="设定加法问题-3.1">设定加法问题</a></span></li><li><span><a href="#预测值初始化" data-toc-modified-id="预测值初始化-3.2">预测值初始化</a></span></li><li><span><a href="#训练神经网络" data-toc-modified-id="训练神经网络-3.3">训练神经网络</a></span><ul class="toc-item"><li><span><a href="#数据读取" data-toc-modified-id="数据读取-3.3.1">数据读取</a></span></li><li><span><a href="#网络训练" data-toc-modified-id="网络训练-3.3.2">网络训练</a></span></li><li><span><a href="#误差分析" data-toc-modified-id="误差分析-3.3.3">误差分析</a></span></li></ul></li><li><span><a href="#进行反向传播" data-toc-modified-id="进行反向传播-3.4">进行反向传播</a></span><ul class="toc-item"><li><span><a href="#数据检索" data-toc-modified-id="数据检索-3.4.1">数据检索</a></span></li><li><span><a href="#误差计算" data-toc-modified-id="误差计算-3.4.2">误差计算</a></span></li><li><span><a href="#进行权值更新" data-toc-modified-id="进行权值更新-3.4.3">进行权值更新</a></span></li></ul></li></ul></li><li><span><a href="#实验总结" data-toc-modified-id="实验总结-4">实验总结</a></span></li></ul></div>

In [1]:
import numpy as np
import copy
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['figure.dpi'] = 96
plt.rcParams['axes.unicode_minus'] = False

## 神经网络层的函数实现

在该实验中将会用到两个用于神经网络层的处理函数， $\sigma$ 函数及其导数。这两个函数用于神经网络层的数据处理， $\sigma$ 函数即是网络中的激活函数。这个函数中具有一些非常优良的以便神经网络使用的特性，这个函数可以将任意值映射到区间 $(0, 1)$ 中，方便将数值和概率相挂钩。

<img align="center" src="../img/01.png">

通过这个函数将数据进行非线性化处理。以便于网络的非线性记忆功能。同时该网络的导数也很容易计算，通过一点的输出值即可求得其导数： $\sigma' = \sigma(1 - \sigma)$ 。

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def diff_sigmoid(x):
    return x * (1 - x)

## 神经网络初始化

### 网络初始化

这里需要将网络的输入层、隐藏层、输出层进行初始化设定，这里我们要预先将神经网络这三层的神经元个数进行设定，以及将网络层与层之间信息交换的权值网络进行初始化设定，同时定义一个和权值网络大小相等的全零网络，用来储存神经网络更新权值的暂存值。

In [3]:
# 步长，输入层，隐含层，输出层
step, input_size, hidden_size, output_size = 0.1, 2, 32, 2

### 权重矩阵 ###
weight_i2h = 2 * np.random.random((input_size, hidden_size)) - 1
weight_h2o = 2 * np.random.random((hidden_size, output_size)) - 1
weight_h2h = 2 * np.random.random((hidden_size, hidden_size)) - 1

### 权重更新（缓存） ###
temp_weight_i2h = np.zeros_like(weight_i2h)
temp_weight_h2o = np.zeros_like(weight_h2o)
temp_weight_h2h = np.zeros_like(weight_h2h)

这里首先初始化三个随机数值的矩阵，分别代表输入层和隐藏层、隐藏层和输出层、隐藏层和隐藏层之间的连接权值，在之后的操作中，这三个矩阵中的数值不断随着网络的学习进行更改，最终使得网络中的权值能正确的对给定的输入值进行处理。

之后网络设定了三个值为零的大小和权值网络相同的矩阵，为的是进行每一次学习的暂时结果的记录。这些矩阵在每一次循环完之后都会将其重置为零矩阵。

### 训练数据集初始化

这里便于神经网络的更精确的计算，将输入神经网络的数值转换成二进制数字。

In [4]:
bin_digits = 8
VALUE_MAX = 2**bin_digits
binary = np.unpackbits(np.arange(VALUE_MAX, dtype=np.uint8).reshape(-1, 1),
                       axis=1)
int2bin = {k: v for k, v in enumerate(binary)}

## 训练神经网络

这里我们将要对网络进行训练，训练的次数越多网络拟合效果越好。在这个实验中我们选定了 20000 次的训练，本小节下面的所有代码，都是在这个 `for` 循环中进行的。

```python
for i in range(20000):
    pass
```

### 设定加法问题

在这一步中，给神经网络出了一个简单的加法问题，目的就是让神经网络通过 20000 次的训练能够学会简单的加法运算，当训练好网络之后，网络进行加减运算时并不是通常意义上的加减，而是通过一步步权值进行处理最终得到正确答案的。这里我们先给出出题代码。

```python
a_decimal = np.random.randint(MAXnumber / 2)
b_decimal = np.random.randint(MAXnumber / 2)
c_decimal = a_decimal + b_decimal
a = i2b[a_decimal]
b = i2b[b_decimal]
c = i2b[c_decimal]
```

首先代码生成了两个随机的数字，并且保证随机数字小于最大数字的一半（ 需要保证两者相加仍然在神经网络的处理范围之内 ）。之后将这两个数字及其加和转化为二进制数字进行储存。

### 预测值初始化

在这一步中，将进行预测值的初始化，设定几个数组，将得到的预测值、误差值、输出层导数进行储存。为了之后训练和反向传播进行准备。

```python
# 网络预测值的二进制数组
binary = np.zeros_like(c)
# 误差值
aError = 0
#导数值
oplayer_der = list()
hdlayer_val = list()
hdlayer_val.append(np.zeros(hdnumber)) 
```

### 训练神经网络

在训练神经网络的过程中首先将数据进行转换，之后数据通过层作用计算权值，并最终将误差进行计算并将权值进行储存以便之后的反向传播运算。在这步中只是做一下网络的训练，层于层之间的网络权值并没有改变。

```python
for locate in range(bdigit):
    X = np.array([[a[bdigit - locate - 1], b[bdigit - locate - 1]]])
    Y = np.array([[c[bdigit - locate - 1]]]).T

    hdlayer = sigmoid(np.dot(X, neu_i2h) + np.dot(hdlayer_val[-1], neu_h2h))
    oplayer = sigmoid(np.dot(hdlayer,neu_h2o))

    oplayer_error = Y - oplayer
    oplayer_der.append((oplayer_error) * dersigmoid(oplayer))
    aError += np.abs(oplayer_error[0])
    binary[bdigit - locate - 1] = np.round(oplayer[0][0])
    hdlayer_val.append(copy.deepcopy(hdlayer))
```

#### 数据读取

```python
X = np.array([[a[bdigit - locate - 1], b[bdigit - locate - 1]]]) #从右向左检索a、b二进制数字
Y = np.array([[c[bdigit - locate - 1]]]).T #检索正确数值
```

这里将二进制数据中的数字从右往左进行读取，并存入到两个数组之中，便于之后的训练。

#### 网络训练

```python
# 隐藏层
hdlayer = sigmoid(np.dot(X, neu_i2h) + np.dot(hdlayer_val[-1], neu_h2h))
# 输出层
oplayer = sigmoid(np.dot(hdlayer, neu_h2o))
```

这段代码在整个网络中起着至关重要的作用。

处理隐藏层的函数是将现在隐藏层的传入信号 $X$ 和输入层与隐藏层的网络权值 `neu_i2h` 进行矩阵相乘，得到现在时刻网络的传入 `np.dot(X,neu_i2h) `。

并和上一时刻网络隐藏层的状态 `np.dot(hdlayer_val[-1], neu_h2h)` （ 上一时刻网络状态的导数 `hdlayer_val[-1]` 和隐藏层与隐藏层的网络权值`neu_h2h` 进行矩阵相乘）进行相加。将加和进行函数处理，得到现在隐藏层的值。

输出层将网络隐藏层得到的值 `hdlayer` 和隐藏层与输出层的网络权值 `neu_h2o` 进行矩阵相乘，得到网络的输出值。

上面两步是网络训练中的关键步骤。请读者结合前文理解清楚。

#### 误差分析

```python
# 计算误差
oplayer_error = Y - oplayer # 真实误差
oplayer_der.append((oplayer_error) * dersigmoid(oplayer)) # 每时刻导数值
aError += np.abs(oplayer_error[0]) # 累加误差的绝对值
```

在这个步骤中，首先计算出输出层和真是值的误差 $oplayer\_error$ ，并将每一次训练输出值的导数进行储存，最后将句对误差进行相加。

### 进行反向传播

到这一步我们得到了神经网络的输出值，下面需要将神经网络进行反向传播来将网络进行进一步的构建。

```python
Fhdlayer_dels = np.zeros(hdnumber)
for locate in range(bdigit):
    X = np.array([[a[locate],b[locate]]])
    hdlayer = hdlayer_val[-locate - 1]
    hdlayer_pre = hdlayer_val[-locate - 2]

    oplayer_dels = oplayer_der[-locate - 1]
    hdlayer_dels = (Fhdlayer_dels.dot(neu_h2h.T) + oplayer_dels.dot(neu_h2o.T)) * dersigmoid(hdlayer)

    neu_h2oN += np.atleast_2d(hdlayer).T.dot(oplayer_dels)
    neu_h2hN += np.atleast_2d(hdlayer_pre).T.dot(hdlayer_dels)
    neu_i2hN += X.T.dot(hdlayer_dels)
    Fhdlayer_dels = hdlayer_dels
```

#### 数据检索

```python
X = np.array([[a[locate], b[locate]]]) # 检索数据
hdlayer = hdlayer_val[-locate-1] # 从数据中取出当前隐藏层
hdlayer_pre = hdlayer_val[-locate-2] # 从数据中取出前一个隐藏层
```

这里反向将数据进行检索，以便于之后的误差的计算。通过前一个循环得到的隐藏层的数组，取出相应的隐藏层的值准备误差运算。

#### 误差计算

```python
# 输出层误差
oplayer_dels = oplayer_der[-locate - 1]
# 隐藏层误差
hdlayer_dels = (Fhdlayer_dels.dot(neu_h2h.T) + oplayer_dels.dot(neu_h2o.T)) * dersigmoid(hdlayer)
```

这里将误差进行统计得到输出层和隐藏层的误差，以便权值进行更新。这里得到了两个网络层的误差，通过这两组误差将权值进行更新。

#### 进行权值更新

```python
neu_i2h += neu_i2hN * step
neu_h2o += neu_h2oN * step
neu_h2h += neu_h2hN * step

neu_i2hN *= 0
neu_h2oN *= 0
neu_h2hN *= 0
```

得到了所有的值之后，将网络的权值进行更新，即是这一次训练对网络权值造成的影响会在这一步中进行更新。

In [5]:
# %% 迭代训练
from tqdm.notebook import tqdm

for i in tqdm(range(20000)):
    ### 构造数据集 ###
    a_dec = np.random.randint(VALUE_MAX / 2)
    b_dec = np.random.randint(VALUE_MAX / 2)
    c_dec = a_dec + b_dec
    a_bin = int2bin[a_dec]
    b_bin = int2bin[b_dec]
    c_bin = int2bin[c_dec]

    # 预测值
    bin_pred = np.zeros_like(c_bin)
    # 误差值
    error_value = 0
    ### 导数 ###
    diff_output_layer = []
    diff_hidden_layer = [np.zeros(hidden_size)]

    ### 训练模型 ###
    for index in range(bin_digits):
        ### 加载数据 ###
        X = np.array(
            [[a_bin[bin_digits - index - 1], b_bin[bin_digits - index - 1]]])
        Y = np.array([[c_bin[bin_digits - index - 1]]]).T
        ### 正向传播 ###
        hidden_layer = sigmoid(
            np.dot(X, weight_i2h) + np.dot(diff_hidden_layer[-1], weight_h2h))
        output_layer = sigmoid(np.dot(hidden_layer, weight_h2o))
        ### 误差分析 ###
        temp_error = Y - output_layer
        diff_output_layer.append(temp_error * diff_sigmoid(temp_error))
        error_value += np.abs(temp_error[0])

        bin_pred[bin_digits - index - 1] = np.round(output_layer[0][0])
        diff_hidden_layer.append(copy.deepcopy(hidden_layer))
        pass

    ### 反向传播 ###
    rev_hidden_layers = np.zeros(hidden_size)
    for index in range(bin_digits):
        ### 数据检索 ###
        X = np.array([[a_bin[index], b_bin[index]]])
        hidden_layer = diff_hidden_layer[-index - 1]
        pre_hidden_layer = diff_hidden_layer[-index - 2]
        ### 计算误差 ###
        output_layer_error = diff_output_layer[-index - 1]
        hidden_layer_error = (
            rev_hidden_layers.dot(weight_h2h.T) +
            output_layer_error.dot(weight_h2o.T)) * diff_sigmoid(hidden_layer)

        temp_weight_h2o += np.atleast_2d(hidden_layer).T.dot(
            output_layer_error)
        temp_weight_h2h += np.atleast_2d(pre_hidden_layer).T.dot(
            hidden_layer_error)
        temp_weight_i2h += X.T.dot(hidden_layer_error)
        rev_hidden_layers = hidden_layer_error
        pass

    ### 权重更新 ###
    weight_i2h += temp_weight_i2h * step
    weight_h2o += temp_weight_h2o * step
    weight_h2h += temp_weight_h2h * step
    temp_weight_h2h *= 0
    temp_weight_h2o *= 0
    temp_weight_i2h *= 0
    pass

  0%|          | 0/20000 [00:00<?, ?it/s]

In [6]:
# %% 模型评估
print("误差值:", str(error_value))
print("预测值:" + str(bin_pred))
print("真实值:" + str(c_bin))
value = 0
for index, x in enumerate(reversed(bin_pred)):
    # 将预测值转换为
    value += x * pow(2, index)
print(str(a_dec) + " + " + str(b_dec) + " = " + str(value))

误差值: [2. 2.]
预测值:[1 1 1 1 1 1 1 1]
真实值:[0 1 1 1 1 1 1 0]
118 + 8 = 255


## 实验总结

本文对 RNN 网络结构进行简单的加法训练，通过 20000 次训练使得网络具有较好的计算能力，实现整数在 $2^8$ 范围内的加法计算。

模型代码逻辑可能存在错误，模型训练结果不理想。